In [30]:
def g_0(n):
    return ("?",)*n

def s_0(n):
    return ("0",)*n



In [31]:
def more_general(h1, h2):
    more_general_parts = []
    for x, y in zip(h1, h2):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
        
    return all(more_general_parts)

l1 = [1,2,3]
l2 = [3,4,5]

list(zip(l1, l2))

[(1, 3), (2, 4), (3, 5)]

In [32]:
def fulfills(example, hypothesis):
    return more_general(hypothesis, example)

def min_generalizations(h, x):
    h_new = list(h)
    for i in range(len(h)):
        if not fulfills(x[i:i+1], h[i:i+1]):
            h_new[i] = "?" if h[i] != "0" else x[i]
            
    return [tuple(h_new)]

In [33]:
#For test
min_generalizations(h=("0", "0", "sunny"), x=("rainy", "windy", "cloudy"))

[('rainy', 'windy', '?')]

In [34]:
def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
                    
        elif h[i] != "0":
            h_new = h[:i] + ("0",) + h[i+1:]
            results.append(h_new)
            
    return results

In [35]:
min_specializations(h=("?", "x",), domains=[["a", "b", "c"], ["x", "y"]], x=("b", "x"))

[('a', 'x'), ('c', 'x'), ('?', '0')]

In [41]:
import csv

with open("enjoySport.csv") as csvFile:
    examples = [tuple(line) for line in csv.reader(csvFile)]
    
examples = examples[1:]
examples

[('Sunny', 'Warm', 'Normal', 'Strong', 'Warn', 'Same ', 'Y'),
 ('Sunny', 'Warm', 'High', 'Strong', 'Warn', 'Same ', 'Y'),
 ('Rainy ', 'Cold', 'High', 'Strong', 'Warn', 'Change', 'N'),
 ('Sunny', 'Warm', 'High', 'Strong', 'Cold', 'Change', 'Y')]

In [42]:
def get_domains(examples):
    d = [set() for i in examples[0]]
    
    for x in examples:
        for i, xi in enumerate(x):
            d[i].add(xi)
    return [list(sorted(x)) for x in d]

get_domains(examples)

[['Rainy ', 'Sunny'],
 ['Cold', 'Warm'],
 ['High', 'Normal'],
 ['Strong'],
 ['Cold', 'Warn'],
 ['Change', 'Same '],
 ['N', 'Y']]

In [43]:
def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
            
        if not fulfills(x, s):
            S.remove(s)
            Splus = min_generalizations(s, x)
            S.update([h for h in Splus if any([more_general(g, h) for g in G])])
            S.difference_update([h for h in S if any([more_general(h, h1) for h1 in S if h != h1])])
            
    return S

In [44]:
def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
            
        if fulfills(x, g):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            G.update([h for h in Gminus if any([more_general(h, s) for s in S])])
            G.difference_update([h for h in G if any([more_general(g1, h) for g1 in G if h != g1])])
            
    return G

In [45]:
domains = get_domains(examples)[:-1]

G = set([g_0(len(domains))])
S = set([s_0(len(domains))])

i=0
print(f'\n S[{i}]: {S}')
print(f'G[{i}]: {G}')

for xcx in examples:
    i = i+1
    xcx = list(xcx)
    
    x,cx = xcx[:-1], xcx[-1]
    if cx=='Y':
        G = {g for g in G if fulfills(x, g)}
        S = generalize_S(x, G, S)
        
    else:
        S = {s for s in S if not fulfills(x, s)}
        G = specialize_G(x, domains, G, S)
        
    print(f'\n S[{i}]: {S}')
    print(f' G[{i}]: {G}')
        


 S[0]: {('0', '0', '0', '0', '0', '0')}
G[0]: {('?', '?', '?', '?', '?', '?')}

 S[1]: {('Sunny', 'Warm', 'Normal', 'Strong', 'Warn', 'Same ')}
 G[1]: {('?', '?', '?', '?', '?', '?')}

 S[2]: {('Sunny', 'Warm', '?', 'Strong', 'Warn', 'Same ')}
 G[2]: {('?', '?', '?', '?', '?', '?')}

 S[3]: {('Sunny', 'Warm', '?', 'Strong', 'Warn', 'Same ')}
 G[3]: {('Sunny', '?', '?', '?', '?', '?'), ('?', 'Warm', '?', '?', '?', '?'), ('?', '?', '?', '?', '?', 'Same ')}

 S[4]: {('Sunny', 'Warm', '?', 'Strong', '?', '?')}
 G[4]: {('Sunny', '?', '?', '?', '?', '?'), ('?', 'Warm', '?', '?', '?', '?')}
